# Load Data
kaggle.com/anirudhg15/mammals-classification

In [ ]:
import pandas as pd
import numpy as np
import os, zipfile, cv2, shutil, math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm as tq
from tensorflow.keras.optimizers import Adam
from keras import optimizers
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Kaggle/

Mounted at /content/drive
/content/drive/MyDrive/Kaggle


In [ ]:
ls

mammals/  Mammals.zip  model_mammals/


In [ ]:
local_zip = '/content/drive/MyDrive/Kaggle/Mammals.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Kaggle')
zip_ref.close()

In [ ]:
os.listdir('/content/drive/MyDrive/Kaggle/mammals')

['bear', 'cat', 'dog', 'elephant', 'goat', 'horse', 'lion', 'tiger', 'wolf']

In [ ]:
from PIL import Image
base_dataset = '/content/drive/MyDrive/Kaggle/mammals'
class_dir = ['bear', 'cat', 'dog', 'elephant', 'goat', 'horse', 'lion', 'tiger', 'wolf']
for class_item in class_dir:
  cur_dir = base_dataset + '/' + class_item + '/'
  dataset = os.listdir(cur_dir)
  for item in dataset :
    im = Image.open(f"{cur_dir}{item}")
    if item.endswith('.jpg'):
      im.save(f"{cur_dir}{item.split('.')[0]}.png", "png")
    else :
      im.save(f"{cur_dir}{item.split('.')[0]}.png", "png")
  for item in dataset :
    if not item.endswith('.png'):
      os.remove(os.path.join(cur_dir, item))

In [ ]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dataset):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [ ]:
# membuat dataframe dari variabel baru
data = pd.DataFrame({'path':full_path,'file_name':file_name,'tag':tag})
print(data.groupby(['tag']).size())

tag
bear        97
cat         99
dog         95
elephant    96
goat        85
horse       67
lion        95
tiger       96
wolf        97
dtype: int64


In [ ]:
x = data['path']
y = data['tag']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.30)

In [ ]:
# menyatukan ke dalam masing-masing dataframe
data_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'training'})

data_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

In [ ]:
print('train size', len(data_tr))
print('val size', len(data_val))

train size 578
val size 249


In [ ]:
dataset_path = '/content/drive/MyDrive/Kaggle/model_mammals'

data_all = data_tr.append([data_tr, data_val]).reset_index(drop=1)

for index, row in tq(data_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

0it [00:00, ?it/s]

In [ ]:
os.listdir('/content/drive/MyDrive/Kaggle/model_mammals/training')

['cat', 'lion', 'goat', 'bear', 'dog', 'elephant', 'horse', 'tiger', 'wolf']

# Preprocessing

In [ ]:
# Gather data train
train_data = []
train_label = []

train_dir='/content/drive/MyDrive/Kaggle/model_mammals/training'
for r, d, f in os.walk(train_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            train_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            train_label.append(label)

train_data = np.array(train_data)
train_label = np.array(train_label)

In [ ]:
# Gather data validation
val_data = []
val_label = []

vali_dir='/content/drive/MyDrive/Kaggle/model_mammals/validation'
for r, d, f in os.walk(vali_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            val_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            val_label.append(label)

val_data = np.array(val_data)
val_label = np.array(val_label)

In [ ]:
print('Data Training Shape : ', train_data.shape)
print('Data Validation Shape : ', val_data.shape)

Data Training Shape :  (578, 250, 250, 3)
Data Validation Shape :  (249, 250, 250, 3)


In [ ]:
# Normalisasi dataset
print("Data sebelum di-normalisasi ", train_data[0][0][0])

x_train = train_data.astype('float32') / 255.0
x_val = val_data.astype('float32') / 255.0
print("Data setelah di-normalisasi ", x_train[0][0][0])

Data sebelum di-normalisasi  [94 73 25]
Data setelah di-normalisasi  [0.36862746 0.28627452 0.09803922]


In [ ]:
# Transformasi label encoder
print('Label sebelum di-encoder ', train_label[567:577])

lb = LabelEncoder()
Y_train = lb.fit_transform(train_label)
Y_val = lb.fit_transform(val_label)

print('Label setelah di-encoder ', Y_train[567:577])

Label sebelum di-encoder  ['wolf' 'wolf' 'wolf' 'wolf' 'wolf' 'wolf' 'wolf' 'wolf' 'wolf' 'wolf']
Label setelah di-encoder  [8 8 8 8 8 8 8 8 8 8]
